# Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

# Images

In [2]:
data_dir_str = "/content/drive/MyDrive/Alternaria"
data_dir = pathlib.Path(data_dir_str)

In [3]:
image_count = len(list(data_dir.glob('*/*.JPG')))
image_count

0

In [ ]:
healthy = list(data_dir.glob('SANAS/*.JPG'))
menor5 = list(data_dir.glob('MENOR A 5_/*.JPG'))
ENTRE5a10 = list(data_dir.glob('5Y10/*.JPG'))
ENTRE10a15 = list(data_dir.glob('10Y15/*.JPG'))
ENTRE15a30 = list(data_dir.glob('15Y30/*.JPG'))
mayor30 = list(data_dir.glob('MAYOR AL 30_/*.JPG'))
print(len(healthy))
print(len(menor5))
print(len(ENTRE5a10))
print(len(ENTRE10a15))
print(len(ENTRE15a30))
print(len(mayor30))

# Standardize image size

In [5]:
batch_size = 10
img_height = 200
img_width = 200

# Training and test split

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.15,
  subset="training",
  seed=2022,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Class names and image labeling



In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
print(type(train_ds))
tensor1,labels1=next(iter(train_ds))

In [ ]:
print(tensor1)
print(labels1)

Label example

In [ ]:
plt.imshow(tensor1[0].numpy().astype("uint8"))
print(class_names[labels1[0]])

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.15,
  subset="validation",
  seed=2022,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Label example

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) #uint8 array de enteros sin signo de 8bits.Inicia en 0
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
testb, testl=next(iter(train_ds))
print(testb.shape)
print(testl.shape)

In [ ]:
normalization_layer = layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image),'   ', np.max(first_image))
plt.imshow(first_image.numpy().astype("uint8"))

Dataset settings

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Architecture of neural networks

In [ ]:
num_classes = 6

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Training accuracy curve must be over validation one, if not the model could be overestimating. As well as, training loss curve must be below the validation one.

To use neural networks you must have a large amount of data, if this is not the case, you could increase the original dataset by rotating the images.

# Data augmentation

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.3),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss') #Training que tan bien se adapta el modelo a los datos de entrenamiento
#validation que tan bien se adapta el modelo a nuevos datos
plt.show()

# Neural network validation using images not used in training or testing.

In [ ]:
img1 = tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/SANAS/DSC_0900.JPG', target_size=(img_height, img_width)
)
img2= tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/MENOR A 5_/DSC_0817.JPG', target_size=(img_height, img_width)
)
img3 = tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/MAYOR AL 30_/DSC_0794.JPG', target_size=(img_height, img_width)
)
img4= tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/5Y10/DSC_0779.JPG', target_size=(img_height, img_width)
)
img5 = tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/15Y30/DSC_0887.JPG', target_size=(img_height, img_width)
)
img6= tf.keras.utils.load_img(
    '/content/drive/MyDrive/Alternaria/10Y15/DSC_0899.JPG', target_size=(img_height, img_width)
)

In [ ]:
img_array1 = tf.keras.utils.img_to_array(img1)
img_array1 = tf.expand_dims(img_array1, 0)

predictions = model.predict(img_array1)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
img_array2 = tf.keras.utils.img_to_array(img2)
img_array2 = tf.expand_dims(img_array2, 0)

predictions = model.predict(img_array2)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
img_array3 = tf.keras.utils.img_to_array(img3)
img_array3 = tf.expand_dims(img_array3, 0)

predictions = model.predict(img_array3)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
) #mayor al 30, acierto

In [ ]:
img_array4 = tf.keras.utils.img_to_array(img4)
img_array4 = tf.expand_dims(img_array4, 0) # Create a batch

predictions = model.predict(img_array4)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
img_array5 = tf.keras.utils.img_to_array(img5)
img_array5 = tf.expand_dims(img_array5, 0)

predictions = model.predict(img_array5)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

In [ ]:
img_array6 = tf.keras.utils.img_to_array(img6)
img_array6 = tf.expand_dims(img_array6, 0)

predictions = model.predict(img_array6)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)